In [ ]:
%env WORKDIR=/tmp/vault
%env PROJECT=hc-73f8ab4df44146ba8cffb466d83
%env KEYRING_NAME=kmse-test
%env KEYRING_GKMS=gcks-test
%env LOCATION=global
%env KEY_NAME=testkey1

In [ ]:
%%bash
cat $WORKDIR/VAULT_TOKEN.TXT
cat $WORKDIR/VAULT_ADDR.TXT

In [ ]:
%env VAULT_ADDR=https://vault-primary-europe-southwest1-jlhb.josemerchan-f986bf.gcp.sbx.hashicorpdemo.com:8200
%env VAULT_TOKEN=hvs.QbtInWOOYqrZJnMrDmcaArKs

# [KMSE](https://developer.hashicorp.com/vault/docs/secrets/key-management) and [KMSE API](https://developer.hashicorp.com/vault/api-docs/secret/key-management#key-management-secrets-engine-api)

In [ ]:
! vault secrets enable keymgmt

In [ ]:
! vault write keymgmt/key/aes256-gcm96 type="aes256-gcm96"
! vault write keymgmt/key/ecdsa-p256 type="ecdsa-p256"

In [ ]:
! vault write keymgmt/key/aes256-key2 type="aes256-gcm96"

In [ ]:
! vault read -format=json keymgmt/key/aes256-gcm96 | jq

## Credentials must be copied to the Vault node and run the command

In [ ]:
! kubectl cp $WORKDIR/creds.json -n vault vault-primary-0:/tmp/creds.json
# ! kubectl cp $WORKDIR/creds.json -n vault vault-primary-1:/tmp/creds.json
# ! kubectl cp $WORKDIR/creds.json -n vault vault-primary-2:/tmp/creds.json
# ! kubectl cp $WORKDIR/creds.json -n vault vault-primary-3:/tmp/creds.json
# ! kubectl cp $WORKDIR/creds.json -n vault vault-primary-4:/tmp/creds.json

[Key compatibility with GCP](https://developer.hashicorp.com/vault/docs/secrets/key-management/gcpkms#key-purpose-compatability) and [GCP Setup Guide](https://developer.hashicorp.com/vault/docs/secrets/gcpkms)

In [ ]:
%%bash

vault write keymgmt/kms/gcpckms \
    provider="gcpckms" \
    key_collection="projects/${PROJECT}/locations/${LOCATION}/keyRings/${KEYRING_NAME}" \
    credentials=service_account_file=/tmp/creds.json

In [ ]:
%%bash

vault write keymgmt/kms/gcpckms/key/aes256-gcm96 \
    purpose="encrypt,decrypt" \
    protection="software"


In [ ]:
%%bash

vault write keymgmt/kms/gcpckms/key/ecdsa-p256 \
    purpose="sign" \
    protection="software"


In [ ]:
%%bash

vault write keymgmt/kms/gcpckms/key/aes256-key2 \
    purpose="encrypt,decrypt" \
    protection="software"

In [ ]:
! vault list keymgmt/kms/gcpckms/key/

In [ ]:
! vault write -f keymgmt/key/aes256-gcm96/rotate

In [ ]:
!  vault read -format=json keymgmt/key/aes256-gcm96 | jq 

> This [engine](https://developer.hashicorp.com/vault/docs/secrets/gcpkms#faq) was created before the functionality of [managed keys](https://developer.hashicorp.com/vault/docs/enterprise/managed-keys)

In [ ]:
# Clean UP
## Remove keys
! vault delete keymgmt/kms/gcpckms/key/aes256-gcm96
! vault delete keymgmt/kms/gcpckms/key/ecdsa-p256
! vault delete keymgmt/kms/gcpckms/key/aes256-key2
! vault delete keymgmt/kms/gcpckms
! vault delete  keymgmt/key/aes256-key2
! vault delete  keymgmt/key/ecdsa-p256
! vault delete  keymgmt/key/aes256-gcm96
! vault secrets disable keymgmt